FRED data: 
- VIX (volatility index): Measures market fear/uncertainty. Spikes during crises.
- 10Y treasury: 
- 2Y treasury:
- Yield curve: (10Y - 2Y) Predicts economic cycles. Inversion often precedes recessions
- BAA corporate Bond: Captures perceived credit risk in corporate bonds. Widens during financial stress.
- CPI: Indicates inflationary environment. Influences policy and real returns.
- Unemployment rate: Reflects slack in the labour market. Lags the business cycle but still valuable.
- Fed Funds Rate: Measures monetary policy stance. Sharp changes can trigger regime shifts.
- GDP growth: measures real economic activity (however it is in quarterly not monthly so may have to forward fill to each corresponding month i.e. fill each missing month with the most recent known GDP value)
- Credit spread (BAA - GS10) (calculated)
- YoY Inflation = 12-month % change in CPI 

API key: 043d7bab55a44c38c28c692cbfddae0e 

In [27]:
import pandas as pd
from datetime import datetime
from fredapi import Fred


In [31]:
fr = Fred(api_key='043d7bab55a44c38c28c692cbfddae0e')# Instantiate FRED with your API key 
df = fr.get_series('UNRATE', observation_start='2006-1-1', frequency = 'm')


In [34]:
df.head()

2006-01-01    4.7
2006-02-01    4.8
2006-03-01    4.7
2006-04-01    4.7
2006-05-01    4.6
dtype: float64

In [ ]:
# Define series of interest
series = {
    'VIX': 'VIXCLS',
    '10Y': 'GS10',
    '2Y': 'GS2',
    'Yield Curve (10Y - 2Y)': 'T10Y2Y',
    'BAA': 'BAA',
    'CPI': 'CPIAUCSL',
    'UnemploymentRate': 'UNRATE',
    'FedFundsRate': 'FEDFUNDS',
    'GDP_YoY': 'A191RL1Q225SBEA'
}